In [35]:
import pandas as pd
from dotenv import load_dotenv
import importlib
pd.set_option("display.max_columns", None)

from shipping_cost_analysis.etl import file_io, preprocessing, db_tools
from shipping_cost_analysis.models.schema import Base

load_dotenv(override=True)

modules_to_reload = [file_io, preprocessing, db_tools]
for module in modules_to_reload:
    importlib.reload(module)

In [ ]:
# import data

file_names = ["dim_customers", "dim_products", "fact_sales_modified", "state_region_mapping"]
dataframes = {}

for file_name in file_names:
    try:
        df = file_io.read_or_write_csv(
            file_name + ".csv",
            file_dir="data",
            to_read=True,
            file_encoding="ISO-8859-1",
            file_sep=","
        )
        dataframes[file_name] = df
    except FileNotFoundError as e:
        print(f"Error loading {file_name}: {e}")

In [31]:
for file in dataframes:
    missing_data = preprocessing.missing_data(dataframes[file])
    if missing_data.shape[0]:
        print(f"{file}:\n {missing_data}\n")

dim_customers:
            Total  Percentage Missing
Latitude      98                2.24
Longitude     98                2.24

fact_sales_modified:
                   Total  Percentage Missing
Customer ID        5434               21.29
Invoice No         1889                7.40
Unit Price         1292                5.06
Quantity           1283                5.03
Sales              1281                5.02
Description        1245                4.88
Stock Code         1225                4.80
Transaction Date   1224                4.79



In [32]:
for file in dataframes:
    print(file)  
    display(dataframes[file].head(3))

dim_customers


,Customer ID,Order City,Order Postal,Order State,Latitude,Longitude
0,18287,Rock Springs,82901-7321,WY,41.5869,-109.2048
1,18283,JACKSON,83001-9460,WY,43.4800,-110.7618
2,18282,Greybull,82426,WY,44.4892,-108.0562


dim_products


,Stock Code,Weight,Landed Cost,Shipping_Cost_1000_mile,Description,Category
0,22423,40.0,35.0,20.0,Taste of the Wild High Prairie Grain-Free Dry ...,Food
1,85123A,2.0,6.0,5.0,Dog and Puppy Pads,Disposables
2,47566,6.0,18.0,12.0,"Memory Foam Pet Beds for Small, Medium, and La...",Disposables


fact_sales_modified


,Transaction Date,Customer ID,Description,Stock Code,Invoice No,Quantity,Sales,Unit Price
0,12/1/2020 8:26,17850.0,Dog and Puppy Pads,85123A,536365.0,1.0,15.30,15.30
1,12/1/2020 8:34,13047.0,Pet Odor Eliminator,84879,536367.0,6.0,60.84,10.14
2,12/1/2020 8:34,13047.0,Rechargeable Pet Nail Grinder,22960,536368.0,1.0,25.50,NaN


state_region_mapping


,ï»¿Order State,State,Region
0,AE,AE,Other
1,ak,AK,Other
2,AK,AK,Other


In [ ]:
customer = dataframes[file_names[0]]
product = dataframes[file_names[1]]
transactions = dataframes[file_names[2]]
state = dataframes[file_names[3]]

In [ ]:
#TODO: 
'''
order_state: Upper + column name
transaction_date: datetime convert
data types: integer -> astype("Int64")
'''

In [ ]:
'''

customers.to_sql("customers", engine, if_exists="replace", index=False)
products.to_sql("products", engine, if_exists="replace", index=False)
orders.to_sql("orders", engine, if_exists="replace", index=False)
order_items.to_sql("order_items", engine, if_exists="replace", index=False)


'''